<a href="https://colab.research.google.com/github/Tranminhtuan48/Object-Detection-Models/blob/main/YOLOv5x_vehicle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Prerequisites & Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
%cd /content/drive/MyDrive
!git clone https://github.com/ultralytics/yolov5

[Errno 2] No such file or directory: '/content/drive/MyDrive'
/content
Cloning into 'yolov5'...
remote: Enumerating objects: 16748, done.
remote: Counting objects: 100% (289/289), done.
remote: Compressing objects: 100% (211/211), done.
remote: Total 16748 (delta 156), reused 159 (delta 78), pack-reused 16459
Receiving objects: 100% (16748/16748), 15.44 MiB | 8.67 MiB/s, done.
Resolving deltas: 100% (11455/11455), done.


In [ ]:
!pip uninstall -y requests
!pip install requests==2.31.0

Found existing installation: requests 2.31.0
Uninstalling requests-2.31.0:
  Successfully uninstalled requests-2.31.0
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)


In [ ]:
!pip install --upgrade google-colab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.1 MB/s eta 0:00:00


In [ ]:
%cd /content/drive/MyDrive/yolov5
%pip install -qr requirements.txt

/content/drive/MyDrive/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.6/799.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 34.4 MB/s eta 0:00:00


In [ ]:
import torch
from drive.MyDrive.yolov5 import utils
display = utils.notebook_init()

YOLOv5 🚀 v7.0-338-gff063284 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.3/78.2 GB disk)


# Buid Dataset

Download dataset:

In [ ]:
# !gdown --id 1J9oDI4Uu27AqFOfdimU4Du4kn-zZzUlp

Ta tiến hành annotation dataset:

- Đối với dataset chưa có labels thì có thể sử dụng roboflow để tiến hành annotation [Roboflow]('https://roboflow.com/annotate?ref=ultralytics'). Mỗi file annotate có yêu cầu như sau:
 - One txt with labels file per image
  - One row per object
  - Each row contains: class_index bbox_x_center bbox_y_center bbox_width bbox_height
  - Box coordinates must be normalized between 0 and 1

VD: `5 0.505000 0.494167 0.920000 0.625000`

- Sau khi annotation xong thì up file vào trong folder yolov5/data như sau:

  - yolov5/data/vehicle/images
    - train
    - val

  - yolov5/data/vehicle/labels
    - train
    - val



####Nếu trường hợp dataset chưa chia train và val thì có thể làm theo như sau:

In [ ]:
import pandas as pd
import os
from shutil import copyfile
df_label_train = pd.read_csv('./Labels/CSV Format/train_labels.csv')
df_label_test = pd.read_csv('./Labels/CSV Format/test_labels.csv')

filename_train = df_label_train ['filename']
filename_test = df_label_test ['filename']

#tạo các thư mục
try:
    os.mkdir('./train_data')
    os.mkdir('./train_data/images')
    os.mkdir('./train_data/labels')
    os.mkdir('./train_data/images/train')
    os.mkdir('./train_data/images/val')
    os.mkdir('./train_data/labels/train')
    os.mkdir('./train_data/labels/val')
except OSError:
    pass

def split_train(SOURCE, DESTINATION_TRAIN,  DESTINATION_TEST):
    for filename in filename_train:
        this_file = SOURCE + filename
        destination = DESTINATION_TRAIN + filename
        copyfile(this_file, destination)

    for filename in filename_test:
        this_file = SOURCE + filename
        destination = DESTINATION_TEST + filename
        copyfile(this_file, destination)

def split_test(SOURCE, DESTINATION_TRAIN,  DESTINATION_TEST):
    for filename in filename_train:
        this_file = SOURCE + filename.replace(".jpg",".txt")
        destination = DESTINATION_TRAIN + filename.replace(".jpg",".txt")
        copyfile(this_file, destination)

    for filename in filename_test:
        this_file = SOURCE + filename.replace(".jpg",".txt")
        destination = DESTINATION_TEST + filename.replace(".jpg",".txt")
        copyfile(this_file, destination)

# #folder images
source_images = './Images/'
destination_images_train = './train_data/images/train/'
destination_images_val = './train_data/images/val/'
split_train(source_images, destination_images_train, destination_images_val)

#folder labels
source_labels = './Labels/TXT format/'
destination_labels_train = './train_data/labels/train/'
destination_labels_val = './train_data/labels/val/'
split_test(source_labels, destination_labels_train, destination_labels_val)


### Tạo file `vehicle.yaml`
  

Tạo một file *vehicle.yaml* trên máy với nội dung như bên dưới rồi upload lên folder *yolov5/data*:

```yaml
train: ../yolov5/data/vehicle/images/train
val: ../yolov5/data/vehicle/images/val # val images (relative to 'path') 128 images
names:
  0: car
  1: threewheel
  2: bus
  3: truck
  4: motorbike
  5: van
  ```

Lưu ý: phần list name của đối tượng phải để lần lượt theo đúng số thứ tự class 0,1,2,3...trong nhãn txt, nếu sai thì khi phát hiện các vật thể gán tên nhãn sẽ bị nhầm với nhau.

#Train


Tiến hành train model với custom dataset. Ta chọn pretrained yolov5 với các thông số phù hợp (cụ thể là chọn v5x -> mạnh nhất):

In [ ]:
# Train YOLOv5
!python train.py --img 640 --batch 16 --epochs 20 --data vehicle.yaml --weights yolov5x.pt

2024-07-09 03:30:11.881393: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-09 03:30:11.881440: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-09 03:30:11.882793: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5x.pt, cfg=, data=vehicle.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, op

Tải ảnh cho vào thư mục ***yolov5/data/images***

Phát hiện đối tượng trên ảnh bằng Script:

In [ ]:
!python detect.py --weights /content/drive/MyDrive/yolov5/runs/train/exp3/weights/best.pt --img 640 --conf 0.25 --source data/images/02973.jpg
display.Image(filename='runs/train/exp3/02973.jpg', width=600)

detect: weights=['/content/drive/MyDrive/yolov5/runs/train/exp3/weights/best.pt'], source=data/images/02973.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-338-gff063284 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 322 layers, 86207059 parameters, 0 gradients, 203.9 GFLOPs
Traceback (most recent call last):
  File "/content/drive/MyDrive/yolov5/detect.py", line 424, in <module>
    main(opt)
  File "/content/drive/MyDrive/yolov5/detect.py", line 419, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 11